# Federated TF Keras TinyImageNet Tutorial
## Using low-level Python API

# Long-Living entities update

* We now may have director running on another machine.
* We use Federation API to communicate with Director.
* Federation object should hold a Director's client (for user service)
* Keeping in mind that several API instances may be connacted to one Director.


* We do not think for now how we start a Director.
* But it knows the data shape and target shape for the DataScience problem in the Federation.
* Director holds the list of connected envoys, we do not need to specify it anymore.
* Director and Envoys are responsible for encrypting connections, we do not need to worry about certs.


* Yet we MUST have a cert to communicate to the Director.
* We MUST know the FQDN of a Director.
* Director communicates data and target shape to the Federation interface object.


* Experiment API may use this info to construct a dummy dataset and a `shard descriptor` stub.

### Install requirements

In [ ]:
!pip install tensorflow==2.6.0

## Connect to the Federation

In [ ]:
# Create a federation
from openfl.interface.interactive_api.federation import Federation

# please use the same identificator that was used in signed certificate
client_id = 'api'
cert_dir = 'cert'
director_node_fqdn = 'localhost'
# 1) Run with API layer - Director mTLS 
# If the user wants to enable mTLS their must provide CA root chain, and signed key pair to the federation interface
# cert_chain = f'{cert_dir}/root_ca.crt'
# api_certificate = f'{cert_dir}/{client_id}.crt'
# api_private_key = f'{cert_dir}/{client_id}.key'

# federation = Federation(client_id=client_id, director_node_fqdn=director_node_fqdn, director_port='50051',
#                        cert_chain=cert_chain, api_cert=api_certificate, api_private_key=api_private_key)

# --------------------------------------------------------------------------------------------------------------------

# 2) Run with TLS disabled (trusted environment)
# Federation can also determine local fqdn automatically
federation = Federation(client_id=client_id, director_node_fqdn=director_node_fqdn, director_port='50051', tls=False)

In [ ]:
shard_registry = federation.get_shard_registry()
shard_registry

## Dataset

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
from openfl.interface.interactive_api.experiment import TaskInterface, DataInterface, ModelInterface, FLExperiment

In [ ]:
import math


def train_transform(img):
    array = tf.keras.preprocessing.image.img_to_array(img)
    if img.mode == 'L':
        original = tf.constant(array)
        array = tf.image.grayscale_to_rgb(original)
    array = tf.reshape(array, array.shape)
    
    return array


def val_transform(img):
    array = tf.keras.preprocessing.image.img_to_array(img)
    if img.mode == 'L':
        original = tf.constant(array)
        array = tf.image.grayscale_to_rgb(original)
    array = tf.reshape(array, array.shape)
    return array


class TransformedDataset(tf.keras.utils.Sequence):
    """Image Person ReID Dataset."""

    def __init__(self, dataset, batch_size, transform=None):
        """Initialize Dataset."""
        self.dataset = dataset
        self.batch_size = batch_size
        self.transform = transform

    def __len__(self):
        """Length of dataset."""
        return math.ceil(len(self.dataset) / self.batch_size)

    def __getitem__(self, index):
        first_id = index * self.batch_size
        last_id = (index + 1) * self.batch_size
        if len(self.dataset) < last_id:
            last_id = len(self.dataset)
    
        batch_x = []
        batch_y = []
        
        for i in range(first_id, last_id):
            img, label = self.dataset[i]
            img = self.transform(img) if self.transform else None
            batch_x.append(img)
            batch_y.append(label)

        return np.array(batch_x), np.array(batch_y)

In [ ]:
class TinyImageNetDataset(DataInterface):
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.train_bs = kwargs['train_bs']
        self.valid_bs = kwargs['valid_bs']
    
    @property
    def shard_descriptor(self):
        return self._shard_descriptor
        
    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.

        This method will be called during a collaborator initialization.
        Local shard_descriptor  will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor
        
        self.train_set = TransformedDataset(
            self._shard_descriptor.get_dataset('train'),
            batch_size=self.train_bs,
            transform=train_transform
        )
        self.valid_set = TransformedDataset(
            self._shard_descriptor.get_dataset('val'),
            batch_size=self.valid_bs,
            transform=val_transform
        )

    def get_train_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks with optimizer in contract
        """
        return self.train_set

    def get_valid_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks without optimizer in contract
        """
        return self.valid_set
    
    def get_train_data_size(self):
        """
        Information for aggregation
        """
        return len(self.train_set)

    def get_valid_data_size(self):
        """
        Information for aggregation
        """
        return len(self.valid_set)

## Model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


num_classes = 200


data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])


# model = Sequential([
# #     layers.experimental.preprocessing.Rescaling(1./255),
# #     data_augmentation,  
#     layers.Conv2D(16, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(32, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(128, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(256, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dense(512, activation='relu'),
#     layers.Dense(num_classes)
# ])

In [ ]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False)
base_model.trainable = False

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(200)


inputs = tf.keras.Input(shape=(64, 64, 3))
#     x = data_augmentation(inputs)
#     x = norm_layer(x)
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
# x = tf.keras.layers.Dropout(0.1)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
train_bs = 32
valid_bs = 32
optimizer = tf.keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
model.build(input_shape=(train_bs, 64, 64, 3))

In [ ]:
model.summary()

In [ ]:
task_interface = TaskInterface()


@task_interface.register_fl_task(model='model', data_loader='train_dataset',
                                 device='device', optimizer='optimizer')     
def train(model, train_dataset, optimizer, device, loss_fn=loss_fn, warmup=False):    
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        y = tf.convert_to_tensor(y_batch_train)
        with tf.GradientTape() as tape:
            y_pred = model(x_batch_train, training=True)  # Forward pass

            loss = loss_fn(y, y_pred)

        # Compute gradients
        trainable_vars = model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update metrics
        train_acc_metric.update_state(y, y_pred)
    
    # Reset training metrics at the end of each epoch
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()
    return {'train_acc': train_acc, 'loss': loss}


@task_interface.register_fl_task(model='model', data_loader='val_dataset', device='device')     
def validate(model, val_dataset, device):
    for x_batch_val, y_batch_val in val_dataset:
        y = tf.convert_to_tensor(y_batch_val)
        # Compute predictions
        y_pred = model(x_batch_val, training=False)
        # Update the metrics.
        val_acc_metric.update_state(y, y_pred)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    
    return {'validation_accuracy': val_acc}

In [ ]:
fed_dataset = TinyImageNetDataset(train_bs=train_bs, valid_bs=valid_bs)

In [ ]:
# Prepare data for local tests
# from tinyimagenet_shard_descriptor import TinyImageNetShardDescriptor


# sd = TinyImageNetShardDescriptor('tinyimagenet_data')
# fed_dataset.shard_descriptor = sd


In [ ]:
# Run local test
# optimizer = tf.keras.optimizers.Adam()
# for i in range(5):
#     print(i)
#     res = train(model=model, optimizer=optimizer, device='CPU', train_dataset=fed_dataset.get_train_loader())
#     print(res)

In [ ]:
# # Prepare data for local tests

# !pip install matplotlib
# import matplotlib.pyplot as plt


# def draw(history, epochs):
#     acc = history.history['accuracy']
#     val_acc = history.history['val_accuracy']

#     loss = history.history['loss']
#     val_loss = history.history['val_loss']

#     epochs_range = range(epochs)

#     plt.figure(figsize=(8, 8))
#     plt.subplot(1, 2, 1)
#     plt.plot(epochs_range, acc, label='Training Accuracy')
#     plt.plot(epochs_range, val_acc, label='Validation Accuracy')
#     plt.legend(loc='lower right')
#     plt.title('Training and Validation Accuracy')

#     plt.subplot(1, 2, 2)
#     plt.plot(epochs_range, loss, label='Training Loss')
#     plt.plot(epochs_range, val_loss, label='Validation Loss')
#     plt.legend(loc='upper right')
#     plt.title('Training and Validation Loss')
#     plt.show()

# IMG_SIZE = 64
# resize_and_rescale = tf.keras.Sequential([
#   layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
# #   layers.experimental.preprocessing.Rescaling(1./255)
# ])
    
# data_augmentation = tf.keras.Sequential([
#   layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
#   layers.RandomCrop(IMG_SIZE, IMG_SIZE),
#   layers.experimental.preprocessing.RandomFlip("horizontal"),
#   layers.experimental.preprocessing.RandomRotation(0.2),
# ])

# norm_layer = tf.keras.layers.Normalization(
#     mean=[0.485, 0.456, 0.406],
#     variance=[0.229, 0.224, 0.225]
# )

# def get_model(lr=0.001):
# #     model = Sequential([
# # #         layers.experimental.preprocessing.Rescaling(1./255, input_shape=(64, 64, 3)),
# #         data_augmentation,
# #         layers.Normalization(mean=[0.485, 0.456, 0.406], variance=[0.229, 0.224, 0.225]),
# #         layers.Conv2D(16, 3, padding='same', activation='relu'),
# #         layers.MaxPooling2D(),
# #         layers.Conv2D(32, 3, padding='same', activation='relu'),
# #         layers.MaxPooling2D(),
# #         layers.Conv2D(64, 3, padding='same', activation='relu'),
# #         layers.MaxPooling2D(),
# #         layers.Conv2D(128, 3, padding='same', activation='relu'),
# #         layers.MaxPooling2D(),
# #         layers.Conv2D(256, 3, padding='same', activation='relu'),
# #         layers.MaxPooling2D(),
# #         layers.Flatten(),
# #         layers.Dense(512, activation='relu'),
# #         layers.Dense(num_classes)
# #     ])

# #     self.model = torchvision.models.mobilenet_v2(pretrained=True)
# #     self.model.requires_grad_(False)
# #     self.model.classifier[1] = torch.nn.Linear(in_features=1280, \
# #                     out_features=200, bias=True)
    
#     base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
#         include_top=False
#     )
#     base_model.trainable = False
    
# # #     base_model.summary()
#     preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
    
#     global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#     prediction_layer = tf.keras.layers.Dense(200)
    
#     inputs = tf.keras.Input(shape=(64, 64, 3))
#     x = data_augmentation(inputs)
# #     x = norm_layer(x)
# #     x = preprocess_input(inputs)
#     x = base_model(x, training=False)
#     x = global_average_layer(x)
#     x = tf.keras.layers.Dropout(0.1)(x)
#     outputs = prediction_layer(x)
#     model = tf.keras.Model(inputs, outputs)

#     optimizer = tf.keras.optimizers.Adam()
#     model.build(input_shape=[train_bs, 64, 64, 3])
#     model.compile(
#         optimizer=optimizer,
#         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#         metrics=['accuracy'],
#     )
#     return model

In [ ]:
# Run local test

# models = {}
# for lr in [0.001]:
#     model = get_model(lr)
#     models[lr] = model
#     epochs=60
#     print(f'{lr=}')
#     history = model.fit(
#       fed_dataset.get_train_loader(),
#       validation_data=fed_dataset.get_valid_loader(),
#       epochs=epochs
#     )
#     draw(history, epochs)

### Register model


In [ ]:
framework_adapter = 'openfl.plugins.frameworks_adapters.keras_adapter.FrameworkAdapterPlugin'
model_interface = ModelInterface(model=model, optimizer=optimizer, framework_plugin=framework_adapter)

## Time to start a federated learning experiment

In [ ]:
# create an experimnet in federation
experiment_name = 'tinyimagenet_test_experiment'
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name)

In [ ]:
# The following command zips the workspace and python requirements to be transfered to collaborator nodes
fl_experiment.start(
    model_provider=model_interface, 
    task_keeper=task_interface,
    data_loader=fed_dataset,
    rounds_to_train=10,
    opt_treatment='CONTINUE_GLOBAL'
)